In [1]:
import numpy as np
import os
import pandas as pd

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'         )
dirIntermediate = os.path.join(dirWork, r'intermediate'  )
dirResults      = os.path.join(dirWork, r'results'       )
dirParams       = os.path.join(dirWork, r'params'        )
dirWebApp       = os.path.join(dirResults, r'webapp_data')

In [3]:
dfTruckData = pd.read_csv(os.path.join(dirIntermediate, r'TruckData_Combined_2012to2019.csv'))
dfSegFac  = pd.read_csv(os.path.join(dirIntermediate, 'SegmentsMatchedToFactorGroups.csv'))
sdfSegments = pd.DataFrame.spatial.from_featureclass(os.path.join(dirWork,  r"arcgis\UDOT-Segments.gdb\Master_Segs_20210811"))

In [4]:
#read in year groups to be able to produce truck factors by different year ranges
prmYearGroups = pd.read_csv(os.path.join(dirParams, r'year_groups.csv'))
prmYearToYearGroups = pd.read_csv(os.path.join(dirParams, r'year_to_year_groups.csv'))

In [5]:
dfSegments = sdfSegments[['SEGID','LRS_ROUTE','BMP','EMP']].copy()

dfSegments['ROUTE']=dfSegments['LRS_ROUTE'].str[0:4]
dfSegments['MP_MID'] = dfSegments['BMP']/2 + dfSegments['EMP']/2

dfSegments = dfSegments.drop(columns=(['LRS_ROUTE']))

#filter by last 5 years
#dfTruckData = dfTruckData[((dfTruckData['YEAR']>=2015) & (dfTruckData['YEAR']<=2019))]

dfTruckDataWithYG = pd.DataFrame.merge(dfTruckData,prmYearToYearGroups,on='YEAR')

dfTruckDataWithYG = dfTruckDataWithYG.drop(columns=(['DESC','SegID']))

dfSegmentsWithTruckData = pd.DataFrame.merge(dfSegments,dfTruckDataWithYG,on='ROUTE')

#filter by segments with midpoint milepost with data range begmp to endmp
dfSegmentsWithTruckData = dfSegmentsWithTruckData[((dfSegmentsWithTruckData['MP_MID']>=dfSegmentsWithTruckData['BEGMP']) & (dfSegmentsWithTruckData['MP_MID']<dfSegmentsWithTruckData['ENDMP']))]

dfSegmentsWithTruckData['VMT']       = dfSegmentsWithTruckData['AADT']  * (dfSegmentsWithTruckData['EMP'] - dfSegmentsWithTruckData['BMP'])
dfSegmentsWithTruckData['SUTRKxVMT'] = dfSegmentsWithTruckData['SUTRK'] * dfSegmentsWithTruckData['VMT']
dfSegmentsWithTruckData['CUTRKxVMT'] = dfSegmentsWithTruckData['CUTRK'] * dfSegmentsWithTruckData['VMT']

dfSegmentsWithTruckDataAggregated = dfSegmentsWithTruckData.groupby(['SEGID','YEARGROUP'],as_index=False).agg(sumVMT=('VMT','sum'),sumSUTRKxVMT=('SUTRKxVMT','sum'),sumCUTRKxVMT=('CUTRKxVMT','sum'))
dfSegmentsWithTruckDataAggregated['SUTRK_wght'] = dfSegmentsWithTruckDataAggregated['sumSUTRKxVMT'] /  dfSegmentsWithTruckDataAggregated['sumVMT']
dfSegmentsWithTruckDataAggregated['CUTRK_wght'] = dfSegmentsWithTruckDataAggregated['sumCUTRKxVMT'] /  dfSegmentsWithTruckDataAggregated['sumVMT']
dfSegmentsWithTruckDataAggregated = dfSegmentsWithTruckDataAggregated[dfSegmentsWithTruckDataAggregated['SUTRK_wght']>0]

dfSegmentsTruck = dfSegmentsWithTruckDataAggregated[['SEGID','YEARGROUP','SUTRK_wght','CUTRK_wght']]
dfSegmentsTruck.columns = ('SEGID','YEARGROUP','SUTRKseg','CUTRKseg')

display(dfSegments)
display(dfTruckDataWithYG)
display(dfSegmentsWithTruckData)
display(dfSegmentsWithTruckDataAggregated)
display(dfSegmentsTruck)


,SEGID,BMP,EMP,ROUTE,MP_MID
0,0163_000.0,0.000,0.458,0163,0.2290
1,3225_000.0,0.000,1.680,3225,0.8400
2,3142_000.0,0.000,0.291,3142,0.1455
3,3149_000.0,0.031,0.289,3149,0.1600
4,0007_006.1,6.125,7.308,0007,6.7165
...,...,...,...,...,...
8552,2080_009.3,9.272,9.636,2080,9.4540
8553,2161_003.2,3.207,3.772,2161,3.4895
8554,WFRC_8220,0.000,0.000,0,0.0000
8555,2161_003.4,3.400,3.772,2161,3.5860


,YEAR,ROUTE,BEGMP,ENDMP,AADT,SUTRK,CUTRK,YEARGROUP
0,2013,0006,0.000,46.010,330.0,0.250000,0.230000,2013
1,2013,0006,0.000,46.010,330.0,0.250000,0.230000,2013-2017
2,2013,0006,0.000,46.010,330.0,0.250000,0.230000,All Years
3,2013,0006,46.010,77.540,340.0,0.180000,0.330000,2013
4,2013,0006,46.010,77.540,340.0,0.180000,0.330000,2013-2017
...,...,...,...,...,...,...,...,...
44859,2019,0491,2.031,17.069,3092.0,0.070659,0.297167,2015-2019
44860,2019,0491,2.031,17.069,3092.0,0.070659,0.297167,All Years
44861,2019,089A,0.000,2.970,5202.0,0.138934,0.060180,2019
44862,2019,089A,0.000,2.970,5202.0,0.138934,0.060180,2015-2019


,SEGID,BMP,EMP,ROUTE,MP_MID,YEAR,BEGMP,ENDMP,AADT,SUTRK,CUTRK,YEARGROUP,VMT,SUTRKxVMT,CUTRKxVMT
0,0163_000.0,0.000,0.458,0163,0.229,2013,0.000,19.250,2080.0,0.250000,0.020000,2013,952.640,238.160000,19.052800
1,0163_000.0,0.000,0.458,0163,0.229,2013,0.000,19.250,2080.0,0.250000,0.020000,2013-2017,952.640,238.160000,19.052800
2,0163_000.0,0.000,0.458,0163,0.229,2013,0.000,19.250,2080.0,0.250000,0.020000,All Years,952.640,238.160000,19.052800
12,0163_000.0,0.000,0.458,0163,0.229,2014,0.000,19.250,2110.0,0.240000,0.020000,2014,966.380,231.931200,19.327600
13,0163_000.0,0.000,0.458,0163,0.229,2014,0.000,19.250,2110.0,0.240000,0.020000,2013-2017,966.380,231.931200,19.327600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732100,0135_000.4,0.364,0.726,0135,0.545,2018,0.398,0.726,33291.0,0.110716,0.124068,2015-2019,12051.342,1334.271933,1495.183448
2732101,0135_000.4,0.364,0.726,0135,0.545,2018,0.398,0.726,33291.0,0.110716,0.124068,All Years,12051.342,1334.271933,1495.183448
2732105,0135_000.4,0.364,0.726,0135,0.545,2019,0.396,0.731,33890.0,0.110700,0.124100,2019,12268.180,1358.087526,1522.481138
2732106,0135_000.4,0.364,0.726,0135,0.545,2019,0.396,0.731,33890.0,0.110700,0.124100,2015-2019,12268.180,1358.087526,1522.481138


,SEGID,YEARGROUP,sumVMT,sumSUTRKxVMT,sumCUTRKxVMT,SUTRK_wght,CUTRK_wght
0,0006_000.0,2013,219.450,54.862500,50.473500,0.250000,0.230000
1,0006_000.0,2013-2017,1239.560,307.456422,285.764972,0.248037,0.230537
2,0006_000.0,2014,232.750,55.860000,53.532500,0.240000,0.230000
3,0006_000.0,2015,249.375,62.343750,57.356250,0.250000,0.230000
4,0006_000.0,2015-2019,1337.315,334.008180,309.595626,0.249760,0.231505
...,...,...,...,...,...,...,...
30475,089A_000.0,2015-2019,56214.253,4392.665339,2895.301243,0.078141,0.051505
30476,089A_000.0,2016,13519.400,811.164000,675.970000,0.060000,0.050000
30477,089A_000.0,2017,14636.220,818.880758,660.765621,0.055949,0.045146
30478,089A_000.0,2019,15288.678,2124.122831,920.067872,0.138934,0.060180


,SEGID,YEARGROUP,SUTRKseg,CUTRKseg
0,0006_000.0,2013,0.250000,0.230000
1,0006_000.0,2013-2017,0.248037,0.230537
2,0006_000.0,2014,0.240000,0.230000
3,0006_000.0,2015,0.250000,0.230000
4,0006_000.0,2015-2019,0.249760,0.231505
...,...,...,...,...
30475,089A_000.0,2015-2019,0.078141,0.051505
30476,089A_000.0,2016,0.060000,0.050000
30477,089A_000.0,2017,0.055949,0.045146
30478,089A_000.0,2019,0.138934,0.060180


In [6]:
#group segments data into Station Groups so that they can be applied to segments that don't have truck data (typically Federal-Aid Routes)

dfSegTruckGroup = pd.DataFrame.merge(dfSegmentsWithTruckData,dfSegFac,on='SEGID')

dfSegTruckGroup = dfSegTruckGroup.drop(columns=(['ROUTE','MP_MID','BEGMP','ENDMP']))

dfSegTruckGroup['VMT2019']   = dfSegTruckGroup['AADT2019'] * (dfSegTruckGroup['EMP'] - dfSegTruckGroup['BMP'])
dfSegTruckGroup['SUTRKxVMT'] = dfSegTruckGroup['SUTRK'] * dfSegTruckGroup['VMT2019']
dfSegTruckGroup['CUTRKxVMT'] = dfSegTruckGroup['CUTRK'] * dfSegTruckGroup['VMT2019']

#aggregate by groups to get values to segments without truck percentage values (typically federal aid and non-state routes)
dfSegTruckGroupAggregated = dfSegTruckGroup.groupby(['STATIONGROUP','YEARGROUP'],as_index=False).agg(sumVMT2019=('VMT2019','sum'),sumSUTRKxVMT=('SUTRKxVMT','sum'),sumCUTRKxVMT=('CUTRKxVMT','sum'))
dfSegTruckGroupAggregated['SUTRK_wght'] = dfSegTruckGroupAggregated['sumSUTRKxVMT'] /  dfSegTruckGroupAggregated['sumVMT2019']
dfSegTruckGroupAggregated['CUTRK_wght'] = dfSegTruckGroupAggregated['sumCUTRKxVMT'] /  dfSegTruckGroupAggregated['sumVMT2019']

dfGroupTruck = dfSegTruckGroupAggregated[['STATIONGROUP','YEARGROUP','SUTRK_wght','CUTRK_wght']]
dfGroupTruck.columns = ('STATIONGROUP','YEARGROUP','SUTRKgrp','CUTRKgrp')

display(dfGroupTruck)

#pd.set_option('display.max_rows', dfGroupTruck.shape[0]+1)
#display(dfGroupTruck)
#pd.set_option('display.max_rows', 10)


,STATIONGROUP,YEARGROUP,SUTRKgrp,CUTRKgrp
0,CO0,2013,0.065502,0.043508
1,CO0,2013-2017,0.063965,0.048449
2,CO0,2014,0.058023,0.045629
3,CO0,2015,0.067624,0.050211
4,CO0,2015-2019,0.078795,0.051343
...,...,...,...,...
945,XX4,2016,0.100425,0.048851
946,XX4,2017,0.073751,0.045944
947,XX4,2018,0.135616,0.037301
948,XX4,2019,0.139055,0.037519


In [7]:
dfGroupTruck['STATIONGROUP'].unique().size

95

In [8]:
dfGroupTruck.groupby(['YEARGROUP'],as_index=False).agg(sgcount=('STATIONGROUP','size'))

,YEARGROUP,sgcount
0,2013,95
1,2013-2017,95
2,2014,95
3,2015,95
4,2015-2019,95
5,2016,95
6,2017,95
7,2018,95
8,2019,95
9,All Years,95


In [9]:
dfSegmentsTruck

,SEGID,YEARGROUP,SUTRKseg,CUTRKseg
0,0006_000.0,2013,0.250000,0.230000
1,0006_000.0,2013-2017,0.248037,0.230537
2,0006_000.0,2014,0.240000,0.230000
3,0006_000.0,2015,0.250000,0.230000
4,0006_000.0,2015-2019,0.249760,0.231505
...,...,...,...,...
30475,089A_000.0,2015-2019,0.078141,0.051505
30476,089A_000.0,2016,0.060000,0.050000
30477,089A_000.0,2017,0.055949,0.045146
30478,089A_000.0,2019,0.138934,0.060180


In [10]:
#create records of all segments with all possible year groups
dfSegFacYG = pd.DataFrame.merge(dfSegFac,prmYearGroups,how='cross')

#add truck data where it already exists for segments
dfSegFacWithTrkSeg = pd.DataFrame.merge(dfSegFacYG,dfSegmentsTruck,on=('SEGID','YEARGROUP'),how='left')

#add truck data where it exists for STATIONGROUP
dfSegFacWithTrkSegTrkGrp = pd.DataFrame.merge(dfSegFacWithTrkSeg,dfGroupTruck,on=('STATIONGROUP','YEARGROUP'),how='left')

#for those without a match, fill with zeros
dfSegFacWithTrkSegTrkGrp = dfSegFacWithTrkSegTrkGrp.fillna(0)

#display
display(dfSegFacWithTrkSegTrkGrp)

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRKseg,CUTRKseg,SUTRKgrp,CUTRKgrp
0,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2013,0.25000,0.230000,0.148692,0.149694
1,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2014,0.24000,0.230000,0.157054,0.149609
2,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2015,0.25000,0.230000,0.162100,0.153160
3,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2016,0.25000,0.230000,0.161335,0.152992
4,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2017,0.24961,0.232449,0.143009,0.152046
...,...,...,...,...,...,...,...,...,...,...,...
256885,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2018,0.00000,0.000000,0.116066,0.053317
256886,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2019,0.00000,0.000000,0.109659,0.048246
256887,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2013-2017,0.00000,0.000000,0.133984,0.052619
256888,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2015-2019,0.00000,0.000000,0.126307,0.053724


In [11]:
dfSegFacWithTrk = dfSegFacWithTrkSegTrkGrp.copy()

dfSegFacWithTrk['SUTRUCKS']=dfSegFacWithTrk['SUTRKseg']
dfSegFacWithTrk['CUTRUCKS']=dfSegFacWithTrk['CUTRKseg']

#if no segment truck, use group truck data
dfSegFacWithTrk.loc[(dfSegFacWithTrk['SUTRKseg']==0),'SUTRUCKS'] = dfSegFacWithTrk['SUTRKgrp']
dfSegFacWithTrk.loc[(dfSegFacWithTrk['CUTRKseg']==0),'CUTRUCKS'] = dfSegFacWithTrk['CUTRKgrp']
display(dfSegFacWithTrk)

dfSegFacWithTrk = dfSegFacWithTrk.drop(columns=(['SUTRKseg','SUTRKgrp','CUTRKseg','CUTRKgrp']))

dfSegFacWithTrk['SUTRUCKS'] = round(dfSegFacWithTrk['SUTRUCKS'],4)
dfSegFacWithTrk['CUTRUCKS'] = round(dfSegFacWithTrk['CUTRUCKS'],4)

dfSegFacWithTrk['AADT2019'] = dfSegFacWithTrk['AADT2019'].astype(int)
dfSegFacWithTrk['AREATYPE'] = dfSegFacWithTrk['AREATYPE'].astype(int)
dfSegFacWithTrk

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRKseg,CUTRKseg,SUTRKgrp,CUTRKgrp,SUTRUCKS,CUTRUCKS
0,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2013,0.25000,0.230000,0.148692,0.149694,0.250000,0.230000
1,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2014,0.24000,0.230000,0.157054,0.149609,0.240000,0.230000
2,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2015,0.25000,0.230000,0.162100,0.153160,0.250000,0.230000
3,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2016,0.25000,0.230000,0.161335,0.152992,0.250000,0.230000
4,0006_000.0,415.0,1.0,Arterial,Statewide,WD0,2017,0.24961,0.232449,0.143009,0.152046,0.249610,0.232449
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256885,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2018,0.00000,0.000000,0.116066,0.053317,0.116066,0.053317
256886,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2019,0.00000,0.000000,0.109659,0.048246,0.109659,0.048246
256887,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2013-2017,0.00000,0.000000,0.133984,0.052619,0.133984,0.052619
256888,WFRC_8223,0.0,4.0,Arterial,Statewide,COU,2015-2019,0.00000,0.000000,0.126307,0.053724,0.126307,0.053724


,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRUCKS,CUTRUCKS
0,0006_000.0,415,1,Arterial,Statewide,WD0,2013,0.2500,0.2300
1,0006_000.0,415,1,Arterial,Statewide,WD0,2014,0.2400,0.2300
2,0006_000.0,415,1,Arterial,Statewide,WD0,2015,0.2500,0.2300
3,0006_000.0,415,1,Arterial,Statewide,WD0,2016,0.2500,0.2300
4,0006_000.0,415,1,Arterial,Statewide,WD0,2017,0.2496,0.2324
...,...,...,...,...,...,...,...,...,...
256885,WFRC_8223,0,4,Arterial,Statewide,COU,2018,0.1161,0.0533
256886,WFRC_8223,0,4,Arterial,Statewide,COU,2019,0.1097,0.0482
256887,WFRC_8223,0,4,Arterial,Statewide,COU,2013-2017,0.1340,0.0526
256888,WFRC_8223,0,4,Arterial,Statewide,COU,2015-2019,0.1263,0.0537


In [12]:
#dfSegFacWithTrk.to_csv(os.path.join(dirIntermediate, r'SegmentTruckFactors.csv'),index=False)
#dfSegFacWithTrk

# after further investigation seen below. going to use 2019 truck data for truck factors
dfSegFacWithTrk_export = dfSegFacWithTrk[dfSegFacWithTrk['YEARGROUP']=='2019']
dfSegFacWithTrk_export = dfSegFacWithTrk_export.drop(columns=('YEARGROUP'))

dfSegFacWithTrk_export.to_csv(os.path.join(dirIntermediate, r'SegmentTruckFactors.csv'),index=False)

display(dfSegFacWithTrk_export)
display(dfSegFacWithTrk_export[dfSegFacWithTrk_export['SEGID']=='WFRC_8223'])

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,SUTRUCKS,CUTRUCKS
6,0006_000.0,415,1,Arterial,Statewide,WD0,0.2496,0.2324
16,0006_000.0,415,1,Arterial,Statewide,SNP,0.2496,0.2324
26,0006_000.0,415,1,Arterial,Statewide,COP,0.2496,0.2324
36,0006_000.7,415,1,Arterial,Statewide,WD0,0.2496,0.2324
46,0006_000.7,415,1,Arterial,Statewide,SNP,0.2496,0.2324
...,...,...,...,...,...,...,...,...
256846,WFRC_8222,0,4,Arterial,Statewide,SNU,0.1097,0.0482
256856,WFRC_8222,0,4,Arterial,Statewide,COU,0.1097,0.0482
256866,WFRC_8223,0,4,Arterial,Statewide,WD3,0.1115,0.0482
256876,WFRC_8223,0,4,Arterial,Statewide,SNU,0.1097,0.0482


,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,SUTRUCKS,CUTRUCKS
256866,WFRC_8223,0,4,Arterial,Statewide,WD3,0.1115,0.0482
256876,WFRC_8223,0,4,Arterial,Statewide,SNU,0.1097,0.0482
256886,WFRC_8223,0,4,Arterial,Statewide,COU,0.1097,0.0482


In [13]:
dfSegFacWithTrk_export[dfSegFacWithTrk_export['SEGID']=='0175_000.0']

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,SUTRUCKS,CUTRUCKS
74526,0175_000.0,32668,3,Arterial,Statewide,WD2,0.2517,0.037
74536,0175_000.0,32668,3,Arterial,Statewide,SNT,0.2517,0.037
74546,0175_000.0,32668,3,Arterial,Statewide,COT,0.2517,0.037


# Investigating Segments that have SU/CU trucks switched in earlier years and fixed in 2019

Segment 0175_000.0 and others on this route appear to have error in pre-2019 data where SUTRUCKS and CUTRUCKS are switched
at least that's what it looks like when considering 2019 and expections.
This route is 11400 South west of State Street

In [14]:
df = dfSegFacWithTrk[dfSegFacWithTrk['SEGID']=='0175_000.0']
df

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,YEARGROUP,SUTRUCKS,CUTRUCKS
74520,0175_000.0,32668,3,Arterial,Statewide,WD2,2013,0.0900,0.2800
74521,0175_000.0,32668,3,Arterial,Statewide,WD2,2014,0.0800,0.2800
74522,0175_000.0,32668,3,Arterial,Statewide,WD2,2015,0.0900,0.2900
74523,0175_000.0,32668,3,Arterial,Statewide,WD2,2016,0.0900,0.2900
74524,0175_000.0,32668,3,Arterial,Statewide,WD2,2017,0.0858,0.2865
74525,0175_000.0,32668,3,Arterial,Statewide,WD2,2018,0.0858,0.2865
74526,0175_000.0,32668,3,Arterial,Statewide,WD2,2019,0.2517,0.0370
74527,0175_000.0,32668,3,Arterial,Statewide,WD2,2013-2017,0.0872,0.2855
74528,0175_000.0,32668,3,Arterial,Statewide,WD2,2015-2019,0.1220,0.2359
74529,0175_000.0,32668,3,Arterial,Statewide,WD2,All Years,0.1122,0.2475


In [15]:
prmYearGroups['YEARGROUP'].tolist()

['2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2013-2017',
 '2015-2019',
 'All Years']

In [16]:
#show only CO groups
dfSU = dfSegFacWithTrk[dfSegFacWithTrk['STATIONGROUP'].str[0:2]=='CO']
dfSU_pivot = dfSU.pivot(index='SEGID', columns='YEARGROUP', values='SUTRUCKS')
dfSU_pivot = dfSU_pivot[prmYearGroups['YEARGROUP'].tolist()]
dfSU_pivot['COMPARE'] = dfSU_pivot['2019'] - dfSU_pivot['2015-2019']
display(dfSU_pivot)

dfCU = dfSegFacWithTrk[dfSegFacWithTrk['STATIONGROUP'].str[0:2]=='CO']
dfCU_pivot = dfCU.pivot(index='SEGID', columns='YEARGROUP', values='CUTRUCKS')
dfCU_pivot = dfCU_pivot[prmYearGroups['YEARGROUP'].tolist()]
dfCU_pivot['COMPARE'] = dfCU_pivot['2019'] - dfCU_pivot['2015-2019']
display(dfCU_pivot)

YEARGROUP,2013,2014,2015,2016,2017,2018,2019,2013-2017,2015-2019,All Years,COMPARE
SEGID,,,,,,,,,,,
0006_000.0,0.2500,0.2400,0.2500,0.2500,0.2496,0.2496,0.2496,0.2480,0.2498,0.2485,-0.0002
0006_000.7,0.2500,0.2400,0.2500,0.2500,0.2496,0.2496,0.2496,0.2480,0.2498,0.2485,-0.0002
0006_016.0,0.2500,0.2400,0.2500,0.2500,0.2496,0.2496,0.2496,0.2480,0.2498,0.2485,-0.0002
0006_046.0,0.1800,0.1700,0.1700,0.1700,0.1751,0.1751,0.1751,0.1728,0.1729,0.1735,0.0022
0006_060.2,0.1800,0.1700,0.1700,0.1700,0.1751,0.1751,0.1751,0.1728,0.1729,0.1735,0.0022
...,...,...,...,...,...,...,...,...,...,...,...
WFRC_8219,0.1282,0.1467,0.1491,0.1428,0.1227,0.1209,0.1278,0.1379,0.1327,0.1340,-0.0049
WFRC_8220,0.1079,0.1280,0.1416,0.1374,0.1080,0.1093,0.1143,0.1246,0.1221,0.1209,-0.0078
WFRC_8221,0.1261,0.1379,0.1468,0.1452,0.1138,0.1161,0.1097,0.1340,0.1263,0.1279,-0.0166


YEARGROUP,2013,2014,2015,2016,2017,2018,2019,2013-2017,2015-2019,All Years,COMPARE
SEGID,,,,,,,,,,,
0006_000.0,0.2300,0.2300,0.2300,0.2300,0.2324,0.2324,0.2324,0.2305,0.2315,0.2311,0.0009
0006_000.7,0.2300,0.2300,0.2300,0.2300,0.2324,0.2324,0.2324,0.2305,0.2315,0.2311,0.0009
0006_016.0,0.2300,0.2300,0.2300,0.2300,0.2324,0.2324,0.2324,0.2305,0.2315,0.2311,0.0009
0006_046.0,0.3300,0.3300,0.3300,0.3300,0.3338,0.3338,0.3338,0.3307,0.3322,0.3316,0.0016
0006_060.2,0.3300,0.3300,0.3300,0.3300,0.3338,0.3338,0.3338,0.3307,0.3322,0.3316,0.0016
...,...,...,...,...,...,...,...,...,...,...,...
WFRC_8219,0.0910,0.0873,0.0915,0.0920,0.0909,0.0904,0.0879,0.0906,0.0905,0.0901,-0.0026
WFRC_8220,0.0668,0.0638,0.0710,0.0687,0.0663,0.0645,0.0577,0.0673,0.0656,0.0655,-0.0079
WFRC_8221,0.0462,0.0498,0.0587,0.0556,0.0528,0.0533,0.0482,0.0526,0.0537,0.0521,-0.0055


In [17]:
import pandas as pd

dfSU_pivot_filtered = dfSU_pivot[dfSU_pivot['COMPARE'].abs()>.05]

def condformatvalue(cell_value):

    highlight_blue = 'background-color: lightblue;'
    default = ''

    if type(cell_value) in [float, int]:
        if abs(cell_value) > .3:
            return highlight_blue

        return default

def condformatdiff(cell_value):

    highlight_yellow = 'background-color: yellow;'
    highlight_orange = 'background-color: orange;'
    highlight_red    = 'background-color: red;'
    default = ''

    if type(cell_value) in [float, int]:
        if abs(cell_value) > .15:
            return highlight_red
        elif abs(cell_value) > .10:
            return highlight_orange
        elif abs(cell_value) > .05:
            return highlight_yellow

        return default

#df.style.applymap(condformat, subset=pd.IndexSlice[:, ['2019ComparedToAllYears']])


(dfSU_pivot_filtered
 .style
 .applymap(condformatdiff, subset=pd.IndexSlice[:, ['COMPARE']])
 .applymap(condformatvalue, subset=pd.IndexSlice[:, ['2017','2018']]))


YEARGROUP,2013,2014,2015,2016,2017,2018,2019,2013-2017,2015-2019,All Years,COMPARE
SEGID,,,,,,,,,,,
0006_173.4,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.158600,0.127000,0.130800,-0.061300
0006_173.7,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.158600,0.127000,0.130800,-0.061300
0006_174.0,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.158400,0.126700,0.130600,-0.061000
0006_174.4,0.070000,0.210000,0.210000,0.220000,0.079400,0.076400,0.065700,0.158400,0.126700,0.130600,-0.061000
0006_177.9,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.227000,0.168600,0.190400,-0.067500
0006_178.8,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.227000,0.168600,0.190400,-0.067500
0006_181.6,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.227000,0.168600,0.190400,-0.067500
0006_184.1,0.250000,0.260000,0.260000,0.260000,0.120600,0.118300,0.101100,0.227000,0.168600,0.190400,-0.067500
0008_000.0,0.330000,0.320000,0.330000,0.330000,0.142400,0.141800,0.132000,0.287800,0.215400,0.246100,-0.083400


In [18]:
#create map1 with zoom set to Salt Lake (any city or place name can be used)
map2 = gis.map('Salt Lake')
map2.basemap='gray'

#join df back to segments

sdfSegmentsWithData = pd.DataFrame.merge(sdfSegments,dfSU_pivot,on='SEGID')

#expression to classify enrollment

arcexp_su = ("var v = $feature.COMPARE;"
             "if      (v<-.12) { return 'neglevel3';}"
             "else if (v<-.08) { return 'neglevel2';}"
             "else if (v<-.04) { return 'neglevel1';}"
             "else if (v< .04) { return '_neutral_';}"
             "else if (v< .08) { return 'poslevel1';}"
             "else if (v< .12) { return 'poslevel2';}"
             "else             { return 'poslevel3';}")

#symbology for enrollment classes
unival_su = [
               {"value":"neglevel3", "label":"Less than -12%", "symbol":{"type":"esriSLS","color":[  0,  0,255,168],"width": 4}},
               {"value":"neglevel2", "label":"-12% to -8%"   , "symbol":{"type":"esriSLS","color":[  0,112,255,168],"width": 3}},
               {"value":"neglevel1", "label":"-8% to -3%"    , "symbol":{"type":"esriSLS","color":[115,178,255,168],"width": 2}},
               {"value":"_neutral_", "label":"-4% to 4%"     , "symbol":{"type":"esriSLS","color":[204,204,204,168],"width": 1}},
               {"value":"poslevel1", "label":"+4% to +8%"    , "symbol":{"type":"esriSLS","color":[255,190,190,168],"width": 2}},
               {"value":"poslevel2", "label":"+8% to +12%"   , "symbol":{"type":"esriSLS","color":[255,127,127,168],"width": 3}},
               {"value":"poslevel3", "label":"More than +12%", "symbol":{"type":"esriSLS","color":[230,  0,  0,168],"width": 4}}
            ]

#define sdf layer
sdfSegmentsWithData.spatial.plot(map_widget = map2,
                                 renderer_type     = 'u-a',
                                 unique_values     = unival_su,
                                 arcade_expression = arcexp_su,
                                 default_symbol    = ""
                                )

map2.layout.height='800px'
map2.legend=True

#add map title
display(Markdown('<h2><center>Segment Single-Unit Changes (2019 vs 2015-2019)</center></h2>'))

#display the map
map2

<h2><center>Segment Single-Unit Changes (2019 vs 2015-2019)</center></h2>

MapView(layout=Layout(height='800px', width='100%'), legend=True)

In [19]:
#create map1 with zoom set to Salt Lake (any city or place name can be used)
map3 = gis.map('Salt Lake')
map3.basemap='gray'

#join df back to segments

sdfSegmentsWithCUData = pd.DataFrame.merge(sdfSegments,dfCU_pivot,on='SEGID')

#expression to classify enrollment

arcexp_su = ("var v = $feature.COMPARE;"
             "if      (v<-.12) { return 'neglevel3';}"
             "else if (v<-.08) { return 'neglevel2';}"
             "else if (v<-.04) { return 'neglevel1';}"
             "else if (v< .04) { return '_neutral_';}"
             "else if (v< .08) { return 'poslevel1';}"
             "else if (v< .12) { return 'poslevel2';}"
             "else             { return 'poslevel3';}")

#symbology for enrollment classes
unival_su = [
               {"value":"neglevel3", "label":"Less than -12%", "symbol":{"type":"esriSLS","color":[  0,  0,255,168],"width": 4}},
               {"value":"neglevel2", "label":"-12% to -8%"   , "symbol":{"type":"esriSLS","color":[  0,112,255,168],"width": 3}},
               {"value":"neglevel1", "label":"-8% to -4%"    , "symbol":{"type":"esriSLS","color":[115,178,255,168],"width": 2}},
               {"value":"_neutral_", "label":"-4% to 4%"     , "symbol":{"type":"esriSLS","color":[204,204,204,168],"width": 1}},
               {"value":"poslevel1", "label":"+4% to +8%"    , "symbol":{"type":"esriSLS","color":[255,190,190,168],"width": 2}},
               {"value":"poslevel2", "label":"+8% to +12%"   , "symbol":{"type":"esriSLS","color":[255,127,127,168],"width": 3}},
               {"value":"poslevel3", "label":"More than +12%", "symbol":{"type":"esriSLS","color":[230,  0,  0,168],"width": 4}}
            ]

#define sdf layer
sdfSegmentsWithCUData.spatial.plot(map_widget        = map3,
                                   renderer_type     = 'u-a',
                                   unique_values     = unival_su,
                                   arcade_expression = arcexp_su,
                                   default_symbol    = ""
                                  )

map3.layout.height='800px'
map3.legend=True

#add map title
display(Markdown('<h2><center>Segment Combo-Unit Changes (2019 vs 2015-2019)</center></h2>'))

#display the map
map3

<h2><center>Segment Combo-Unit Changes (2019 vs 2015-2019)</center></h2>

MapView(layout=Layout(height='800px', width='100%'), legend=True)

In [20]:
sdfSegmentsWithData

,OBJECTID,SEGID,BMP,EMP,LRS_ROUTE,ROUTE_ID,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,...,2014,2015,2016,2017,2018,2019,2013-2017,2015-2019,All Years,COMPARE
0,1,0163_000.0,0.000,0.458,0163P,163,HWY 163,Arterial,5,Arterial,...,0.2400,0.2500,0.2500,0.2458,0.2458,0.2458,0.2472,0.2474,0.2467,-0.0016
1,2,3225_000.0,0.000,1.680,3225P,3225,UTAH AVE,Arterial,6,Arterial,...,0.1512,0.1546,0.1558,0.1499,0.1503,0.1494,0.1524,0.1520,0.1517,-0.0026
2,3,3142_000.0,0.000,0.291,3142P,3142,RIVER RD,Arterial,4,Arterial,...,0.1854,0.1883,0.1883,0.1261,0.1274,0.1282,0.1744,0.1517,0.1611,-0.0235
3,4,3149_000.0,0.031,0.289,3149P,7,DESERT CANYONS PKWY,Arterial,2,Freeway,...,0.1854,0.1883,0.1883,0.1261,0.1274,0.1282,0.1744,0.1517,0.1611,-0.0235
4,5,0007_006.1,6.125,7.308,0007P,7,SOUTHERN PKWY,Arterial,2,Freeway,...,0.3200,0.3200,0.3200,0.1528,0.1541,0.1556,0.2695,0.1977,0.2185,-0.0421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8481,8554,2080_009.3,9.272,9.636,2080P,2080,SEGO LILY DR,Arterial,5,Arterial,...,0.1280,0.1416,0.1374,0.1080,0.1093,0.1143,0.1246,0.1221,0.1209,-0.0078
8482,8557,2161_003.2,3.207,3.772,2161P,2161,NEW BINGHAM HWY,Arterial,3,Arterial,...,0.1280,0.1416,0.1374,0.1080,0.1093,0.1143,0.1246,0.1221,0.1209,-0.0078
8483,8560,WFRC_8220,0.000,0.000,0,0,AIRPORT RD,Arterial,4,Arterial,...,0.1280,0.1416,0.1374,0.1080,0.1093,0.1143,0.1246,0.1221,0.1209,-0.0078
8484,8561,2161_003.4,3.400,3.772,2161P,2161,7800 S,Arterial,4,Arterial,...,0.1280,0.1416,0.1374,0.1080,0.1093,0.1143,0.1246,0.1221,0.1209,-0.0078
